# Get data

In [1]:
!wget --no-check-certificate https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip -O ./Data/rps.zip
  
!wget --no-check-certificate https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip -O ./Data/rps-test-set.zip

--2021-05-18 21:46:50--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.195.176, 142.250.195.240, 142.250.71.16, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.195.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 200682221 (191M) [application/zip]
Saving to: ‘./Data/rps.zip’

./Data/rps.zip      100%[===================>] 191.38M  2.26MB/s    in 82s     

2021-05-18 21:48:13 (2.33 MB/s) - ‘./Data/rps.zip’ saved [200682221/200682221]

--2021-05-18 21:48:13--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.195.48, 142.250.195.112, 142.250.196.80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.195.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29516758 (28M) [application/zip]
Saving t

In [4]:
import os
import zipfile
import shutil 

local_zip = './Data/rps.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./Data/')
zip_ref.close()
os.remove('./Data/rps.zip')

local_zip = './Data/rps-test-set.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./Data/')
zip_ref.close()
os.remove('./Data/rps-test-set.zip')

# Data generator

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(rescale=1./255)
train_datagen = train_gen.flow_from_directory(
    "./Data/rps",
    target_size=(150,150),
    batch_size=32,
    class_mode='categorical'
)

val_gen = ImageDataGenerator(rescale=1./255)
val_datagen = val_gen.flow_from_directory(
    "./Data/rps-test-set",
    target_size=(150,150),
    batch_size=32,
    class_mode='categorical'
)

Found 2520 images belonging to 3 classes.
Found 372 images belonging to 3 classes.


# Model defination

In [8]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(len(os.listdir('./Data/rps/')), activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)             0

In [9]:
model.compile(
    optimizer='adam',
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
history = model.fit_generator(
    train_datagen,
    validation_data=val_datagen,
    epochs=2,
)

In [ ]:
import  matplotlib.pyplot as plt

no_of_epochs = len(history.history["accuracy"])
epochs = range(no_of_epochs)
plt.plot(epochs, history.history["accuracy"], label="Training")
plt.plot(epochs, history.history["val_accuracy"], label="Validation")
plt.title("Training and validation accuracy")
plt.legend()

plt.figure()
plt.plot(epochs, history.history["loss"], label="Training")
plt.plot(epochs, history.history["val_loss"], label="Validation")
plt.title("Training and validation loss")
plt.legend()